# 🎤 Piper Voice Showcase

Interactive demo of **4 different Piper TTS voices** including LibriTTS with 904 speakers.
Switch between voice models and speakers with autoplay.

**Available Voices:**
- 🎙️ **Heather** (default) - Natural, conversational female
- 🎙️ **Mike** - Balanced, natural male  
- 🎙️ **Lessac** - Professional, high quality
- 🎙️ **LibriTTS** - 904 diverse speakers

**Demo Content:**
- 📄 Text loaded from `demo.md`
- 🔧 Preprocessed with `tts_preprocessing.py`

**Requirements:**
- `piper-tts`
- `ipywidgets`
- `tts_preprocessing.py` module
- `demo.md` file

**Usage:** Run all cells → `create_player()`

**Output:** Generated audio files saved to `wav/` folder

In [ ]:
"""
Piper Voice Showcase - Jupyter (FIXED AUTOPLAY)
"""

from pathlib import Path
import urllib.request
import wave

from piper import PiperVoice, SynthesisConfig
from ipywidgets import Dropdown, Output, VBox, Button, HBox
from IPython.display import Audio, display, clear_output
from tts_preprocessing import preprocess_markdown_for_tts

# ============================================================================
# CONFIG
# ============================================================================

MODELS_DIR = Path("models/piper")
WAV_DIR = Path("wav")
WAV_DIR.mkdir(exist_ok=True)

# Load demo text from file
DEMO_FILE = Path("demo.md")

if not DEMO_FILE.exists():
    raise FileNotFoundError(
        f"Demo file not found: {DEMO_FILE}\n"
        "Please ensure demo.md is in the same directory as this notebook."
    )

with open(DEMO_FILE, 'r', encoding='utf-8') as f:
    TEXT = f.read()

TEST_TEXT = preprocess_markdown_for_tts(TEXT)

print(f"✓ Loaded demo text from: {DEMO_FILE}")
print(f"  Original length: {len(TEXT)} chars")
print(f"  Preprocessed length: {len(TEST_TEXT)} chars")
print()

AVAILABLE_VOICES = {
    "heather": {
        "name": "Heather",
        "model": "en_US-hfc_female-medium",
        "description": "Natural, conversational female voice",
        "size": "medium",
        "quality": "medium",
        "speakers": 1
    },
    "mike": {
        "name": "Mike",
        "model": "en_US-hfc_male-medium",
        "description": "Balanced, natural male voice",
        "size": "medium",
        "quality": "medium",
        "speakers": 1
    },
    "lessac": {
        "name": "Lessac",
        "model": "en_US-lessac-high",
        "description": "Professional, natural speech. Best quality.",
        "size": "high",
        "quality": "high",
        "speakers": 1
    },
    "libritts": {
        "name": "LibriTTS",
        "model": "en_US-libritts-high",
        "description": "904 diverse speakers (high quality)",
        "size": "high",
        "quality": "high",
        "speakers": 904
    }
}

# ============================================================================
# DOWNLOAD MODEL
# ============================================================================

def download_model(voice_key: str):
    MODELS_DIR.mkdir(parents=True, exist_ok=True)
    
    voice_info = AVAILABLE_VOICES[voice_key]
    model_name = voice_info["model"]
    
    onnx_path = MODELS_DIR / f"{model_name}.onnx"
    json_path = MODELS_DIR / f"{model_name}.onnx.json"

    if onnx_path.exists() and json_path.exists():
        return onnx_path

    print(f"Downloading {voice_info['name']} ({model_name})...")
    
    # Construct HuggingFace URL based on model name
    parts = model_name.split("-")
    lang = parts[0]  # e.g., en_US
    voice_name = "-".join(parts[1:-1])  # e.g., hfc_female, libritts
    quality = parts[-1]  # e.g., medium, high
    
    base_url = (
        f"https://huggingface.co/rhasspy/piper-voices/"
        f"resolve/main/en/{lang}/{voice_name}/{quality}"
    )

    urllib.request.urlretrieve(f"{base_url}/{model_name}.onnx", onnx_path)
    urllib.request.urlretrieve(f"{base_url}/{model_name}.onnx.json", json_path)

    return onnx_path

# ============================================================================
# LOAD MODEL
# ============================================================================

loaded_voices = {}

def load_voice(voice_key: str):
    if voice_key in loaded_voices:
        return loaded_voices[voice_key]
    
    print(f"Loading {AVAILABLE_VOICES[voice_key]['name']}...")
    model_path = download_model(voice_key)
    voice = PiperVoice.load(str(model_path))
    loaded_voices[voice_key] = voice
    print(f"✓ {AVAILABLE_VOICES[voice_key]['name']} loaded ({voice.config.sample_rate} Hz)")
    return voice

# ============================================================================
# SYNTHESIS → WAV FILE
# ============================================================================

def synthesize_to_wav(voice_key: str, speaker_id: int, wav_path: Path):
    voice = load_voice(voice_key)
    syn_config = SynthesisConfig(speaker_id=speaker_id)

    pcm = bytearray()
    for chunk in voice.synthesize(TEST_TEXT, syn_config=syn_config):
        pcm.extend(chunk.audio_int16_bytes)

    with wave.open(str(wav_path), "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(voice.config.sample_rate)
        f.writeframes(pcm)

def get_wav_filename(voice_key: str, speaker_id: int = 0) -> Path:
    """Generate consistent filename for voice/speaker combination"""
    voice_info = AVAILABLE_VOICES[voice_key]
    if voice_info["speakers"] > 1:
        return WAV_DIR / f"{voice_key}_speaker_{speaker_id}.wav"
    else:
        return WAV_DIR / f"{voice_key}.wav"

# ============================================================================
# GENERATE SAMPLE FILES
# ============================================================================

def generate_sample_files():
    """Generate the 4 main sample files for README"""
    print("\n🎵 Generating sample files for README...")
    
    samples_to_generate = [
        ("heather", 0),
        ("mike", 0),
        ("lessac", 0),
        ("libritts", 0),
    ]
    
    for voice_key, speaker_id in samples_to_generate:
        wav_path = get_wav_filename(voice_key, speaker_id)
        if not wav_path.exists():
            print(f"  Creating {wav_path.name}...")
            synthesize_to_wav(voice_key, speaker_id, wav_path)
        else:
            print(f"  ✓ {wav_path.name} already exists")
    
    print("✅ Sample files ready!\n")

# ============================================================================
# PLAYER UI
# ============================================================================

def create_player():
    # Voice model dropdown (main selector for 4 voices)
    voice_dropdown = Dropdown(
        options=[
            (f"{v['name']} - {v['description']}", k) 
            for k, v in AVAILABLE_VOICES.items()
        ],
        value="heather",
        description="Voice:",
        layout={"width": "500px"},
    )

    # Speaker controls (ONLY for LibriTTS)
    speaker_dropdown = Dropdown(
        options=[(f"Speaker {i}", i) for i in range(904)],
        value=0,
        description="Speaker:",
        layout={"width": "300px"},
    )

    prev_button = Button(
        description="−",
        layout={"width": "40px"},
        tooltip="Previous speaker",
    )

    next_button = Button(
        description="+",
        layout={"width": "40px"},
        tooltip="Next speaker",
    )

    speaker_controls = HBox([prev_button, speaker_dropdown, next_button])
    speaker_controls.layout.display = "none"  # Hidden by default
    
    output = Output()

    def play_voice(voice_key: str, speaker_id: int = 0):
        with output:
            clear_output(wait=True)
            voice_info = AVAILABLE_VOICES[voice_key]
            
            wav_path = get_wav_filename(voice_key, speaker_id)
            
            if voice_key == "libritts":
                print(f"🎤 {voice_info['name']} - Speaker {speaker_id}")
            else:
                print(f"🎤 {voice_info['name']}")
            
            print(f"💾 {wav_path.name}")

            # Generate if doesn't exist
            if not wav_path.exists():
                synthesize_to_wav(voice_key, speaker_id, wav_path)
            
            display(Audio(filename=str(wav_path), autoplay=True))

    def update_ui_for_voice(voice_key: str):
        """Show speaker selector only for LibriTTS"""
        if voice_key == "libritts":
            speaker_controls.layout.display = "flex"
            speaker_dropdown.value = 0
        else:
            speaker_controls.layout.display = "none"

    def on_voice_change(change):
        if change["name"] == "value":
            voice_key = change["new"]
            update_ui_for_voice(voice_key)
            speaker_id = speaker_dropdown.value if voice_key == "libritts" else 0
            play_voice(voice_key, speaker_id)

    def on_speaker_change(change):
        if change["name"] == "value":
            play_voice(voice_dropdown.value, change["new"])

    def on_prev_clicked(_):
        if voice_dropdown.value == "libritts" and speaker_dropdown.value > 0:
            speaker_dropdown.value -= 1

    def on_next_clicked(_):
        if voice_dropdown.value == "libritts" and speaker_dropdown.value < 903:
            speaker_dropdown.value += 1

    voice_dropdown.observe(on_voice_change)
    speaker_dropdown.observe(on_speaker_change)
    prev_button.on_click(on_prev_clicked)
    next_button.on_click(on_next_clicked)

    # Initial setup
    update_ui_for_voice("heather")
    play_voice("heather", 0)

    display(VBox([voice_dropdown, speaker_controls, output]))

# ============================================================================
# READY
# ============================================================================

# Generate sample files on load
generate_sample_files()

print("✅ Piper Voice Showcase ready!")
print(f"📁 Audio files saved to: {WAV_DIR.absolute()}")
print("Run: create_player()\n")

In [ ]:
create_player()